In [ ]:
!pip install transformers --upgrade

In [ ]:

!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import openai
import pandas as pd
from datasets import Dataset
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import random
from sklearn.model_selection import train_test_split
import copy
random.seed(42)
import re # for extract tuples
from itertools import combinations # for generate combinations

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 25 06:13:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
with open("acs_overall_not_synthesis_paragraph.txt",'r',encoding='utf-8') as f:
  not_synthesis = f.read()

with open("acs_overall_synthesis_paragraph.txt",'r',encoding='utf-8') as f:
  is_synthesis = f.read()



In [ ]:
df_test = pd.read_csv("Training_SynParas.csv")
df_test = df_test[['Paragraph','if_synthesis']]
df_test = df_test.rename(columns={'Paragraph': 'text', 'if_synthesis': 'label'})
df_test = df_test.dropna()
df_test = df_test.reset_index()

In [ ]:
training_no_syn = [record.rsplit("\t",1)[0] for record in not_synthesis.split('\n')]
training_syn = [record.rsplit("\t",2)[0] for record in is_synthesis.split('\n')]

# down-sampling non-synthesis paragraphs to 50-50
random.shuffle(training_no_syn)
training_no_syn = training_no_syn[:len(training_syn)]


training_labels = []
training_labels += [0]*len(training_no_syn)
training_labels += [1]*len(training_syn)
training_texts = training_no_syn + training_syn

In [ ]:
df_train = pd.DataFrame(columns=['text','label'])
df_train['text'] = training_texts
df_train['label'] = training_labels


In [ ]:
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [ ]:
#df_train.to_csv("training_data.csv",index=False)

In [ ]:
dataset = Dataset.from_pandas(df_train)

In [ ]:
# 2. Tokenize the text
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=500)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

# 3. Load pre-trained BERT for classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)




[Widget output removed for GitHub preview]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_dataset_test = Dataset.from_pandas(df_test).map(tokenize_function, batched=True)

tokenized_dataset_test = tokenized_dataset_test.rename_column("label", "labels")
tokenized_dataset_test.set_format("torch")

In [ ]:
# 4. Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    # Remove or comment out 'evaluation_strategy'
    # evaluation_strategy="epoch", # This line is causing the error
    logging_dir='./logs',
    learning_rate=1e-5,
    report_to="none"  # Explicitly disable wandb reporting
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_test
)

trainer.train()



Step,Training Loss
500,0.141800
1000,0.092000
1500,0.067400
2000,0.044200
2500,0.033600


TrainOutput(global_step=2688, training_loss=0.07260230591609365, metrics={'train_runtime': 447.9522, 'train_samples_per_second': 47.992, 'train_steps_per_second': 6.001, 'total_flos': 5523790496220000.0, 'train_loss': 0.07260230591609365, 'epoch': 3.0})

In [ ]:
# 6. Save your model
model.save_pretrained("./bert_binary_syn_classifier")
tokenizer.save_pretrained("./bert_binary_syn_classifier")

('./bert_binary_syn_classifier/tokenizer_config.json',
 './bert_binary_syn_classifier/special_tokens_map.json',
 './bert_binary_syn_classifier/vocab.txt',
 './bert_binary_syn_classifier/added_tokens.json')

In [ ]:
from transformers import TextClassificationPipeline


In [ ]:

# 1. Load model and tokenizer
model_path = "./bert_binary_syn_classifier"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# 2. Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Create the pipeline manually
classifier = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    return_all_scores=True  # optional, shows probs for all labels
)



Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
def get_binary_label(prediction_list):
    top_label = max(prediction_list, key=lambda x: x['score'])['label']
    return 1 if top_label == 'LABEL_1' else 0



In [ ]:
# 4. Run predictions
val_texts = df_test['text'].tolist()
val_labels = df_test['label'].tolist()

pred_labels = classifier(val_texts,truncation=True, max_length=500)

In [ ]:
pred_labels = [get_binary_label(pred) for pred in pred_labels]

In [ ]:
from sklearn.metrics import classification_report

target_names = ['Not Synthesis', 'Is Synthesis']
print(classification_report(val_labels, pred_labels, target_names=target_names))

               precision    recall  f1-score   support

Not Synthesis       0.95      0.95      0.95       170
 Is Synthesis       0.88      0.88      0.88        69

     accuracy                           0.93       239
    macro avg       0.92      0.92      0.92       239
 weighted avg       0.93      0.93      0.93       239



In [ ]:
from google.colab import files
!zip -r ./bert_binary_syn_classifier.zip ./bert_binary_syn_classifier # Create a zip archive of the folder


  adding: bert_binary_syn_classifier/ (stored 0%)
  adding: bert_binary_syn_classifier/config.json (deflated 49%)
  adding: bert_binary_syn_classifier/vocab.txt (deflated 53%)
  adding: bert_binary_syn_classifier/special_tokens_map.json (deflated 42%)
  adding: bert_binary_syn_classifier/tokenizer_config.json (deflated 75%)
  adding: bert_binary_syn_classifier/model.safetensors (deflated 7%)


In [ ]:
files.download('./bert_binary_syn_classifier.zip') # Download the zip file


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import time

start = time.time()
# --- Line of code to measure ---
test = classifier(val_texts,truncation=True, max_length=500)
end = time.time()

print("Runtime:", end - start, "seconds")

Runtime: 2.9798362255096436 seconds


## Relation Extraction

In [ ]:
# merge text paragraph to structured action graph
with open("acs_overall_clear_output1225.txt",'r',encoding='utf-8') as f:
  acs_overall_clear_output = f.read()

with open("codified_synthesis_action_graph.txt",'r',encoding='utf-8') as f:
  codified_synthesis_action_graph = f.read()


In [ ]:
list_paragraphs = acs_overall_clear_output.split('\n')
list_paragraphs[0].split('\t',maxsplit=2)
para_idx = []
para_text = []
for paragraph in list_paragraphs:
  if paragraph.strip() == '':
    continue
  para_idx.append(paragraph.split('\t',maxsplit=2)[2])
  para_text.append(paragraph.split('\t',maxsplit=2)[1])
assert len(para_idx) == len(para_text)

In [ ]:
list_graphs = codified_synthesis_action_graph.split('END\n')
graph_idx = []
graph_relations = []
num_error = 0
for graph in list_graphs:
  if graph.strip() == '':
    continue
  try:
    graph_idx.append(graph.rsplit('\t',maxsplit=1)[0])
    graph_relations.append(graph.rsplit('\t',maxsplit=1)[1].split('Entity-entity pairs related with each step, delimited by comma:\n')[1])
  except:
    num_error += 1
    print(graph)
    continue
assert len(graph_idx) == len(graph_relations)

In [ ]:
df_paragraphs = pd.DataFrame(columns=['idx','text'])
df_paragraphs['idx'] = para_idx
df_paragraphs['text'] = para_text

df_graphs = pd.DataFrame(columns=['idx','relations'])
df_graphs['idx'] = graph_idx
df_graphs['relations'] = graph_relations

# prompt: inner join df_paragraphs and df_graphs based on column 'idx'

df_merged = pd.merge(df_paragraphs, df_graphs, on='idx', how='inner')

## Generate Training Dataset for Relation Extraction

In [ ]:
pattern = r'\(([^,]+),\s*([^)]+)\)'
list_tuples = []
for relations in df_merged['relations']:
  # Find all matches and return as list of tuples
  list_tuples += [re.findall(pattern, relations)]



In [ ]:
combo_tuples = []
combo_labels = []
for tuples in list_tuples:
  tuple_set = set(tuples)

  # Flatten the list of tuples and create a set to remove duplicates
  unique_items = set([item for pair in tuples for item in pair])

  # Generate all unique 2-element combinations (unordered)
  combinations_list = list(combinations(unique_items, 2))

  combo_tuples += [combinations_list]

  # Check if each combination or its reverse exists in original tuples
  exists_flags = []
  for a, b in combinations_list:
      if (a, b) in tuple_set or (b, a) in tuple_set:
          exists_flags.append(1)
      else:
          exists_flags.append(0)
  combo_labels += [exists_flags]


In [ ]:
list_original_texts = df_merged['text'].tolist()

In [ ]:
labeled_texts_for_training = []
for index in range(len(list_original_texts)):
  original_text = list_original_texts[index]
  # List of entity pairs to label
  target_pairs = combo_tuples[index]

  # Store the labeled versions
  labeled_versions = []

  for chem, detail in target_pairs:
    text = copy.deepcopy(original_text)

    # Escape both parts for regex safety
    chem_pattern = re.escape(chem)
    detail_pattern = re.escape(detail)

    # Replace the first occurrence of chem
    text = re.sub(chem_pattern, f"<ENT>{chem}</ENT>", text, count=1)

    # Replace the first occurrence of detail
    text = re.sub(detail_pattern, f"<ENT>{detail}</ENT>", text, count=1)

    # Save result
    labeled_versions.append(text)
  labeled_texts_for_training += [labeled_versions]


assert len(labeled_texts_for_training) == len(combo_labels)

### Flatten both text and label list

In [ ]:
final_training_texts = []
final_labels = []
for index in range(len(labeled_texts_for_training)):
  final_training_texts += labeled_texts_for_training[index]
  final_labels += combo_labels[index]
assert len(final_training_texts) == len(final_labels)

In [ ]:
# gathering everything into one single data frame
df = pd.DataFrame(columns=['text','label'])
df['text'] = final_training_texts
df['label'] = final_labels

## Upsampling Training Set

While upsampling the training dataset, the test dataset is untouched

In [ ]:
# Step 1: Train-test split (stratified to preserve label distribution)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Step 2: Separate majority and minority classes in the training set
train_majority = train_df[train_df['label'] == 0]
train_minority = train_df[train_df['label'] == 1]

# Step 3: Downsample the majority class to match the size of the minority class
train_majority_downsampled = train_majority.sample(n=len(train_minority), random_state=42)

# Step 4: Combine and shuffle
train_balanced = pd.concat([train_majority_downsampled, train_minority], ignore_index=True)
train_balanced = train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Final datasets:
# - train_balanced: training set with balanced classes (via downsampling)
# - test_df: test set with original class distribution
# Now you have:
# - train_balanced: training data with balanced classes
# - test_df: untouched test set with original class distribution


## Start Training Experiment

In [ ]:
# 1. convert pandas data frame into DataSet format
dataset_train = Dataset.from_pandas(train_balanced)
dataset_test = Dataset.from_pandas(test_df)

# 2. Tokenize the text
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=500)

tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_train = tokenized_dataset_train.rename_column("label", "labels")
tokenized_dataset_train.set_format("torch")

tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)
tokenized_dataset_test = tokenized_dataset_test.rename_column("label", "labels")
tokenized_dataset_test.set_format("torch")

# 3. Load pre-trained BERT for classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Widget output removed for GitHub preview]

[Widget output removed for GitHub preview]

[Widget output removed for GitHub preview]

[Widget output removed for GitHub preview]

[Widget output removed for GitHub preview]

[Widget output removed for GitHub preview]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


[Widget output removed for GitHub preview]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 4. Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    eval_strategy="epoch",
    weight_decay=0.01,
    warmup_steps=500,
    # Remove or comment out 'evaluation_strategy'
    # evaluation_strategy="epoch", # This line is causing the error
    logging_dir='./logs',
    learning_rate=2e-5,
    report_to="none"  # Explicitly disable wandb reporting
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted'),
        'precision': precision_score(labels, preds, average='weighted', zero_division=0),
        'recall': recall_score(labels, preds, average='weighted')
    }

In [ ]:
# training starts
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    compute_metrics=compute_metrics
)
trainer.train()



Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.250000,0.301549,0.884312,0.910965,0.960084,0.884312
2,0.205500,0.272479,0.889304,0.914526,0.961901,0.889304
3,0.172200,0.266146,0.892414,0.916696,0.962693,0.892414
4,0.133000,0.252912,0.923452,0.937704,0.965324,0.923452


TrainOutput(global_step=16544, training_loss=0.21869189022703373, metrics={'train_runtime': 8642.0201, 'train_samples_per_second': 30.624, 'train_steps_per_second': 1.914, 'total_flos': 6.800187447984e+16, 'train_loss': 0.21869189022703373, 'epoch': 4.0})

In [ ]:
# 6. Save your model
model.save_pretrained("./bert_relation_classifier")
tokenizer.save_pretrained("./bert_relation_classifier")

('./bert_relation_classifier/tokenizer_config.json',
 './bert_relation_classifier/special_tokens_map.json',
 './bert_relation_classifier/vocab.txt',
 './bert_relation_classifier/added_tokens.json')

In [ ]:
from google.colab import files
!zip -r ./bert_relation_classifier.zip ./bert_relation_classifier # Create a zip archive of the folder
files.download('./bert_relation_classifier.zip') # Download the zip file


  adding: bert_relation_classifier/ (stored 0%)
  adding: bert_relation_classifier/vocab.txt (deflated 53%)
  adding: bert_relation_classifier/model.safetensors (deflated 7%)
  adding: bert_relation_classifier/config.json (deflated 49%)
  adding: bert_relation_classifier/special_tokens_map.json (deflated 42%)
  adding: bert_relation_classifier/tokenizer_config.json (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.2529124617576599, 'eval_accuracy': 0.9234517184551816, 'eval_f1': 0.9377040771157283, 'eval_precision': 0.9653242369578728, 'eval_recall': 0.9234517184551816, 'eval_runtime': 925.4658, 'eval_samples_per_second': 180.964, 'eval_steps_per_second': 5.656, 'epoch': 4.0}
